#### Merge the datasets

In [3]:
# !pip install pandas

In [13]:
import pandas as pd

df_sentiment = pd.read_csv("../data/all_banks_reviews_with_sentiment.csv")
df_themes = pd.read_csv("../data/all_banks_reviews_with_themes.csv")

assert len(df_sentiment) == len(df_themes), "Row count mismatch!"

df_sentiment["keywords"] = df_themes["keywords"]
df_sentiment["identified_themes"] = df_themes["identified_themes"]

df_sentiment.to_csv("../data/all_bank_reviews.csv", index=False)

#### Connect to Oracle

In [18]:
import oracledb

conn = oracledb.connect(
    user="system",
    password="oracle@1234",
    dsn="localhost/XEPDB1"
)
cursor = conn.cursor()

In [19]:
for table in ['reviews', 'banks']:
    try:
        cursor.execute(f"DROP TABLE {table} CASCADE CONSTRAINTS")
        print(f"Dropped existing table: {table}")
    except oracledb.DatabaseError as e:
        print(f"Table {table} did not exist or couldn't be dropped:", e)

Table reviews did not exist or couldn't be dropped: ORA-00942: table or view does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/
Table banks did not exist or couldn't be dropped: ORA-00942: table or view does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/


#### Create the Tables

In [20]:
# Create banks table
cursor.execute("""
CREATE TABLE banks (
    id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    name VARCHAR2(100) NOT NULL UNIQUE
)
""")

# create reviews table
cursor.execute("""
CREATE TABLE reviews (
    id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    review CLOB,
    rating NUMBER,
    review_date DATE,
    bank VARCHAR2(100),
    source VARCHAR2(50),
    review_corrected CLOB,
    bert_sentiment VARCHAR2(20),
    sentiment_score FLOAT,
    adjusted_sentiment VARCHAR2(20),
    keywords CLOB,
    identified_themes CLOB
)
""")

conn.commit()
cursor.close()
conn.close()

print(" All tables created successfully.")


 All tables created successfully.


In [21]:
# !pip install pandas

In [24]:
def clean_val(val):
    if pd.isna(val):
        return None
    return val

In [26]:
import pandas as pd
# Load your enhanced review data
df = pd.read_csv("../data/all_bank_reviews.csv")

# Connect to Oracle
conn = oracledb.connect(
    user="system",
    password="oracle@1234",
    dsn="localhost/XEPDB1"
)
cursor = conn.cursor()

# Insert data into the reviews table

for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO reviews (
            review, rating, review_date, bank, source,
            review_corrected, bert_sentiment, sentiment_score,
            adjusted_sentiment, keywords, identified_themes
        ) VALUES (
            :1, :2, TO_DATE(:3, 'YYYY-MM-DD'), :4, :5,
            :6, :7, :8, :9, :10, :11
        )
    """, (
        row['review'],
        int(row['rating']),
        row['date'],   # date is already a string like 'YYYY-MM-DD'
        row['bank'],
        row['source'],
        row['review_corrected'],
        row['bert_sentiment'],
        float(row['sentiment_score']),
        row['adjusted_sentiment'],
        clean_val(row['keywords']),         # convert NaN -> None here
        row['identified_themes']
    ))


conn.commit()
cursor.close()
conn.close()

print("All data inserted successfully into reviews table.")


All data inserted successfully into reviews table.
